<a href="https://colab.research.google.com/github/m1lle3r/university/blob/main/Extra_tasks/Ex_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Дополнительное задание №2.2. Замыкания. Декораторы. Итераторы. Генераторы**

**БАЗА:**

- **Замыкания** позволяют создавать функции с сохраняющимся состоянием. Это полезно для создания фабричных функций и функций с настраиваемым поведением.
- **Декораторы** позволяют модифицировать или расширять поведение функций без изменения их исходного кода.

---

## **I. Замыкания и декораторы**

### **Пункт №1**

Напишите две функции создания списка из чётных чисел от 0 до N (N – аргумент функции): \([0, 2, 4, ..., N]\).

- **Первая функция** должна использовать метод `append` для добавления элементов в список.
- **Вторая функция** должна использовать **генератор списков** (list comprehensions) для создания списка.

После этого, через **декоратор**, определите время работы этих функций.

In [ ]:
# Ваш код
import time

def measure_time(func):
    def wrapper(*args):
        start_time = time.time()
        result = func(*args)
        print(f'Время выполнения {time.time() - start_time}')
        return result
    return wrapper

@measure_time
def list_comprehension(N: int) -> list:
    return [number for number in range(0, N, 2)]

@measure_time
def create_list(N: int) -> list:
    even_numbers = []
    for number in range(0, N, 2):
        even_numbers.append(number)
    return even_numbers

print(create_list(10))
print(list_comprehension(10))

Время выполнения 3.814697265625e-06
[0, 2, 4, 6, 8]
Время выполнения 5.245208740234375e-06
[0, 2, 4, 6, 8]


---

### **Пункт №2**

Напишите **декоратор** для кэширования результатов работы функции, вычисляющей значение n-го числа [**ряда Фибоначчи**](https://ru.wikipedia.org/wiki/Числа_Фибоначчи).

То есть, при повторном вызове функции через декоратор уже имеющийся результат должен браться из кэша, а не вычисляться заново.

**Например:**

- При значении параметра `n = 5`, должна кэшироваться последовательность \([0, 1, 1, 2, 3, 5]\).
- Вызывая после этого целевую функцию через декоратор ещё раз с `n = 3`, результат \([0, 1, 1, 2]\) должен браться из кэша.
- Если последующее значение `n` больше предыдущего, например `n = 10`, вычисление должно продолжаться, начиная с закэшированной последовательности.

*Подсказка: используйте **замыкание** для хранения кэша внутри декоратора.*


In [ ]:
def fibonacci(n: int) -> int:
    if n in [1, 2]:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

def index_of_fibonacci(value: int) -> int:
    index = 0
    while fibonacci(index) != value:
        index += 1
    return index

def memoize(func):
    cache = [None] * 100

    def wrapper(n: int):
        if n < 1:
            return None
        if cache[n] is not None:
            return cache[n]

        cache[n] = func(n)
        return cache[n]

    return wrapper

@memoize
def fibonacci_with_cache(n: int) -> int:
    if n in [1, 2]:
        return 1
    return fibonacci_with_cache(n - 1) + fibonacci_with_cache(n - 2)

print(fibonacci_with_cache(1))
print(fibonacci_with_cache(10))

1
55


---

### **Пункт №3**

Примените к функции из задания №2 сразу **два декоратора**:

1. **Декоратор**, определяющий время выполнения функции.
2. **Кэширующий декоратор** (из задания №2).

Сравните время работы функции с использованием кэширования и без него.


In [ ]:
import time

def fibonacci(n: int) -> int:
    if n in [1, 2]:
        return 1
    a, b = 1, 1
    for _ in range(3, n + 1):
        a, b = b, a + b
    return b

def index_of_fibonacci(value: int) -> int:
    index = 1
    while True:
        fib_value = fibonacci(index)
        if fib_value == value:
            return index
        if fib_value > value:
            return -1
        index += 1

def memoize(func):
    cache = {}

    def wrapper(n: int):
        if n < 1:
            return None
        if n in cache:
            return cache[n]
        cache[n] = func(n)
        return cache[n]

    return wrapper

def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        print(f'Время выполнения: {time.time() - start_time:.6f} секунд')
        return result
    return wrapper

@memoize
@measure_time
def fibonacci_with_cache(n: int) -> int:
    if n in [1, 2]:
        return 1
    return fibonacci_with_cache(n - 1) + fibonacci_with_cache(n - 2)

@measure_time
def fibonacci_without_cache(n: int) -> int:
    if n in [1, 2]:
        return 1
    a, b = 1, 1
    for _ in range(3, n + 1):
        a, b = b, a + b
    return b

print("Без кэширования:")
print(fibonacci_without_cache(30))

print("\nС кэшированием:")
print(fibonacci_with_cache(30))

Без кэширования:
Время выполнения: 0.000010 секунд
832040

С кэшированием:
Время выполнения: 0.000002 секунд
Время выполнения: 0.000001 секунд
Время выполнения: 0.000301 секунд
Время выполнения: 0.000343 секунд
Время выполнения: 0.000368 секунд
Время выполнения: 0.000386 секунд
Время выполнения: 0.000416 секунд
Время выполнения: 0.000466 секунд
Время выполнения: 0.000494 секунд
Время выполнения: 0.006574 секунд
Время выполнения: 0.006618 секунд
Время выполнения: 0.006650 секунд
Время выполнения: 0.006679 секунд
Время выполнения: 0.006714 секунд
Время выполнения: 0.006747 секунд
Время выполнения: 0.009172 секунд
Время выполнения: 0.009215 секунд
Время выполнения: 0.009243 секунд
Время выполнения: 0.009266 секунд
Время выполнения: 0.009294 секунд
Время выполнения: 0.009319 секунд
Время выполнения: 0.009465 секунд
Время выполнения: 0.009628 секунд
Время выполнения: 0.009660 секунд
Время выполнения: 0.009688 секунд
Время выполнения: 0.014532 секунд
Время выполнения: 0.014786 секунд
Время в

---

### **Пункт №4**

Создайте функцию **make_multiplier(n)**, которая принимает число **n** и возвращает функцию, умножающую переданное ей число на **n**.

**Пример использования:**

```python
def make_multiplier(n):
    # Ваш код

times3 = make_multiplier(3)
print(times3(5))  # Вывод: 15
```

In [ ]:
# Ваш код
def make_multiplier(n):
    def inner(value):
      return n*value
    return inner

multiplier = make_multiplier(3)
print(multiplier(5))

15


---

### **Пункт №5**

Реализуйте функцию с замыканием, которая настраивает округление чисел до заданного количества знаков после запятой.

**Пример:**

```python
def rounder(n):
    # Ваш код

round_to_2 = rounder(2)
print(round_to_2(3.14159))  # Вывод: 3.14
```


In [ ]:
# Ваш код
def rounder(n):
  def inner(value):
        return round(value, n)
  return inner

round_to_2 = rounder(2)
print(round_to_2(3.14159))

3.14


---

### **Пункт №6**

Напишите **декоратор**, который измеряет время исполнения функции и выводит его на экран, только если время превышает определённый порог.

**Пример:**

```python
@time_threshold(threshold=0.5)
def long_running_function():
    # Долгий код

long_running_function()
# Выводится время выполнения только если оно больше 0.5 секунд
```

In [ ]:
# Ваш код
import time

def execution_threshold(threshold):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            func(*args, **kwargs)
            if time.time() - start_time > threshold:
                print(f'Время выполнения {time.time() - start_time}')
        return wrapper
    return decorator

@execution_threshold(threshold=0.5)
def long_running_task():
    result = []
    for num in range(10**8):
        result.append(num * 0.5)

long_running_task()

Время выполнения 18.51417899131775


---

## **II. Итераторы и генераторы**

---

### **Пункт №1. Генератор строк фиксированной длины**

Напишите генератор `string_generator(char, times)`, который генерирует строки, состоящие из символа `char`, повторенного от 1 до `times` раз.

```python
# Пример использования:
for s in string_generator('*', 5):
    print(s)
# Вывод:
# *
# **
# ***
# ****
# *****
```



---

In [ ]:
# Ваш код
def custom_string_generator(base_string: str = '', times: int = 1):
    index = 0
    while index < times:
        yield base_string
        base_string += '*'
        index += 1

for item in custom_string_generator('*', 5):
    print(item)

*
**
***
****
*****


---

### **Пункт №2. Генератор бесконечной последовательности**

Создайте бесконечный генератор `infinite_sequence()`, который с каждым вызовом возвращает следующее число, начиная с 1.

```python
# Пример использования:
gen = infinite_sequence()
for _ in range(5):
    print(next(gen))
# Вывод:
# 1
# 2
# 3
# 4
# 5
```

---

In [ ]:
# Ваш код
def infinite_sequence():
    number = 1
    while True:
        yield number
        number += 1

gen = infinite_sequence()
for _ in range(5):
    print(next(gen))

1
2
3
4
5


---

### **Пункт №3. Генератор комбинированных списков**

Создайте генератор `combined_lists(lst1, lst2)`, который попеременно возвращает элементы из `lst1` и `lst2`. Если длины списков неравны, генератор должен остановиться при исчерпании более короткого списка.

```python
# Пример использования:
for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)
# Вывод:
# 1
# 'a'
# 2
# 'b'
# 3
# 'c'
```

---

In [ ]:
# Ваш код
def combined_lists(lst1, lst2):
    min_length = min(len(lst1), len(lst2))
    for i in range(min_length):
        yield lst1[i]
        yield lst2[i]

for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)

1
a
2
b
3
c


---

### **Пункт №4. Генератор перевернутой строки**

Напишите генератор `reverse_string(s)`, который при каждом вызове возвращает следующий символ строки `s` в обратном порядке.

```python
# Пример использования:
for char in reverse_string('hello'):
    print(char)
# Вывод:
# o
# l
# l
# e
# h
```

---

In [ ]:
# Ваш код
def reverse_string(s):
    for char in reversed(s):
        yield char

for char in reverse_string('hello'):
    print(char)

o
l
l
e
h


---

### **Пункт №5. Генератор степеней двойки**

Создайте генератор `powers_of_two(n)`, который возвращает степени двойки от 2^0 до 2^n.

```python
# Пример использования:
for num in powers_of_two(5):
    print(num)
# Вывод:
# 1  # 2^0
# 2  # 2^1
# 4  # 2^2
# 8  # 2^3
# 16 # 2^4
# 32 # 2^5
```

---

In [ ]:
# Ваш код
def powers_of_two(n):
    for i in range(n + 1):
        yield 2 ** i

for num in powers_of_two(5):
    print(num)

1
2
4
8
16
32


---

### **Пункт №6. Генератор чисел из строки**

Напишите генератор `number_extractor(s)`, который извлекает числа из заданной строки `s` и возвращает их как целые числа.

```python
# Пример использования:
for num in number_extractor('abc123def45gh6'):
    print(num)
# Вывод:
# 123
# 45
# 6
```

---

In [ ]:
# Ваш код
def number_extractor(s):
    number = ''
    for char in s:
        if char.isdigit():
            number += char
        elif number:
            yield int(number)
            number = ''
    if number:
        yield int(number)

for num in number_extractor('abc123def45gh6'):
    print(num)

123
45
6


---